You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
result = crew.kickoff(inputs={"topic": "Data Science"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Data Science.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: Unveiling the Latest Trends and Key Players in Data Science

Outline:
I. Introduction
- Brief overview of the importance of data science in today's digital age
- Mention of the rapid advancements and innovations in the field

II. Latest Trends in Data Science
A. Artificial Intelligence and Machine Learning
- Explanation of how AI and ML are shaping the future of data science
- Examples of real-world applications and success stories

B. Big Data Analytics
- Discussion on the challenges and opportunities of handli

I now can give a great answer

Final Answer:
# Unveiling the Latest Trends and Key Players in Data Science

In today's digital age, data science plays a crucial role in driving innovation and decision-making across various industries. With the rapid advancements in technology, the field of data science continues to evolve, paving the way for new trends and key players to emerge.

## Latest Trends in Data Science

### Artificial Intelligence and Machine Learning
Artificial Intelligence (AI) and Machine Learning (ML) have revolutionized the way data is analyzed and utilized. From predictive analytics to natural language processing, AI and ML algorithms are reshaping the future of data science. Real-world applications such as personalized recommendations in e-commerce and fraud detection in finance showcase the power of AI and ML in transforming data into actionable insights.

### Big Data Analytics
Handling large volumes of data has become a common challenge for organizations, giving ris

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Latest Trends and Key Players in Data Science

In today's digital age, data science plays a crucial role in driving innovation and decision-making across various industries. With the rapid advancements in technology, the field of data science continues to evolve, paving the way for new trends and key players to emerge.

## Latest Trends in Data Science

### Artificial Intelligence and Machine Learning
Artificial Intelligence (AI) and Machine Learning (ML) have revolutionized the way data is analyzed and utilized. From predictive analytics to natural language processing, AI and ML algorithms are reshaping the future of data science. Real-world applications such as personalized recommendations in e-commerce and fraud detection in finance showcase the power of AI and ML in transforming data into actionable insights.

### Big Data Analytics
Handling large volumes of data has become a common challenge for organizations, giving rise to the need for sophisticated big data analytics tools and technologies. From Apache Hadoop to Spark, these tools enable businesses to extract valuable information from massive datasets. Despite the challenges, big data analytics presents opportunities for organizations to gain a competitive edge through data-driven decision-making.

### Data Privacy and Ethics
As data collection becomes more pervasive, concerns around data privacy and ethical considerations in data science have come to the forefront. Companies are now facing increased scrutiny over their data practices, leading to a greater emphasis on ethical data handling. Case studies highlighting the repercussions of unethical data practices serve as a reminder of the importance of maintaining data privacy and ethical standards in the field of data science.

## Key Players in Data Science

### Companies Leading the Way
Top tech companies such as Google, Amazon, and Microsoft are at the forefront of investing in data science to drive innovation and gain a competitive advantage. Their contributions to the field, from developing cutting-edge algorithms to leveraging cloud computing for data analysis, have reshaped industry trends and set new standards for data-driven decision-making.

### Influential Data Scientists
Renowned data scientists like Fei-Fei Li, Geoffrey Hinton, and Yann LeCun have made significant contributions to the field of data science through their groundbreaking research and innovations. Their work in areas such as computer vision, deep learning, and neural networks has played a pivotal role in shaping the future of data science and inspiring the next generation of data enthusiasts.

## Noteworthy News in Data Science

### Recent Developments
Recent news and developments in the data science industry continue to drive innovation and shape the future of the field. From advancements in AI and ML to breakthroughs in big data analytics, these events underscore the dynamic nature of data science and highlight the importance of staying updated on emerging trends and key developments.

## Target Audience Analysis

Professionals in the tech industry, data enthusiasts, and students pursuing careers in data science form the core audience for staying updated on trends, key players, and news in the field. Understanding their interests and pain points, such as keeping pace with rapid changes in data science and navigating the competitive landscape, is essential for delivering valuable insights and resources to support their professional growth.

## SEO Keywords

- Data Science Trends 2021
- Top Data Science Companies
- Ethical Data Practices
- Big Data Analytics Tools
- AI and ML Innovations

## Resources

For the latest news and updates in data science, consider exploring reputable websites and publications dedicated to the field. Research studies and whitepapers on emerging trends in data science provide valuable insights for professionals looking to stay ahead of the curve. Additionally, interviews and podcasts featuring key players in the data science industry offer a firsthand look at the latest innovations and developments shaping the future of the field.

**Call to Action:**
Stay ahead of the curve in the world of data science by subscribing to our newsletter for regular updates on the latest trends, key players, and news in the industry.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).